# 콜백
##### Langchain은 여러 단계에 연결할 수 있는 콜백 시스템을 제공한다.
##### 로깅, 모니터링, 스트리밍 및 기타 작업에 유용하다.

## 콜백 핸들러
##### CallbackHandlers는 구독할 수 있는 각 이벤트에 대한 메서드가 있는 CallbackHandler 인터페이스를 구현하는 개체이다.
##### CallbackManager는 이벤트가 트리거될 때 각 핸들러에서 적절한 메서드를 호출한다.

```python
class BaseCallbackHandler:
    """Base callback handler that can be used to handle callbacks from langchain."""

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        """Run when LLM starts running."""

    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs: Any
    ) -> Any:
        """Run when Chat Model starts running."""

    def on_llm_new_token(self, token: str, **kwargs: Any) -> Any:
        """Run on new LLM token. Only available when streaming is enabled."""

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""

    def on_llm_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """Run when LLM errors."""

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> Any:
        """Run when chain starts running."""

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs: Any) -> Any:
        """Run when chain ends running."""

    def on_chain_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """Run when chain errors."""

    def on_tool_start(
        self, serialized: Dict[str, Any], input_str: str, **kwargs: Any
    ) -> Any:
        """Run when tool starts running."""

    def on_tool_end(self, output: Any, **kwargs: Any) -> Any:
        """Run when tool ends running."""

    def on_tool_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """Run when tool errors."""

    def on_text(self, text: str, **kwargs: Any) -> Any:
        """Run on arbitrary text."""

    def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
        """Run on agent action."""

    def on_agent_finish(self, finish: AgentFinish, **kwargs: Any) -> Any:
        """Run on agent end."""
```

## 콜백 핸들러 예제
##### 가장 기본적인 핸들러인 StdOutCallbackHandler(모든 이벤트를 기록) 사용해보자.
##### verbose 가 True 인 경우 StdOutCallbackHandler를 명시적으로 전달하지 않아도 호출된다.

```python
from langchain_core.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate

handler = StdOutCallbackHandler()
llm = OpenAI()
prompt = PromptTemplate.from_template("1 + {number} = ")

# 생성자 콜백
chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
chain.invoke({"number":2})

# verbose flag 사용
chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
chain.invoke({"number":2})

# 요청 콜백
chain = LLMChain(llm=llm, prompt=prompt)
chain.invoke({"number":2}, {"callbacks":[handler]})
```



*   생성자 콜백 : 생성자에 정의되는데, 이 경우 콜백은 해당 객체에 대한 모든 호출에 사용되며 해당 객체로만 범위가 지정된다. - 단일 요청이 아닌 전체 체인에 해당하는 로깅, 모니터링에 주로 사용
*   요청 콜백 : 요청을 발행하는 데 사용되는 "invoke" 메소드에 정의된다. 이 경우 콜백은 해당 특정 요청과 여기에 포함된 모든 하위 요청에만 사용된다. - 단일 요청의 스트리밍에 주로 사용



## 비동기 콜백
##### 비동기 API를 사용하려는 경우 AsyncCallbackHandler 를 사용하는 것이 권장된다.
##### 비동기 방법을 사용하는 동안 동기화를 사용하면 CallbackHandler가 사용되지만, 내부적으로 문제를 일으킬 수 있는 호출이 수행된다. (run_in_executor)

```python
import asyncio
from typing import Any, Dict, List

from langchain.callbacks.base import AsyncCallbackHandler, BaseCallbackHandler
from langchain_core.messages import HumanMessage
from langchain_core.outputs import LLMResult
from langchain_openai import ChatOpenAI


class MyCustomSyncHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"Sync handler being called in a `thread_pool_executor`: token: {token}")


class MyCustomAsyncHandler(AsyncCallbackHandler):
    """Async callback handler that can be used to handle callbacks from langchain."""

    async def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> None:
        """Run when chain starts running."""
        print("zzzz....")
        await asyncio.sleep(0.3)
        class_name = serialized["name"]
        print("Hi! I just woke up. Your llm is starting")

    async def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        """Run when chain ends running."""
        print("zzzz....")
        await asyncio.sleep(0.3)
        print("Hi! I just woke up. Your llm is ending")

# 스트리밍을 사용하기 위해, chatmodel 생성자에 streaming=True 옵션을 줘야한다.
# 추가적으로 사용자 정의 핸들러를 수행해야 한다.
chat = ChatOpenAI(
    max_tokens=25,
    streaming=True,
    callbacks=[MyCustomSyncHandler(), MyCustomAsyncHandler()],
)

await chat.agenerate([[HumanMessage(content="Tell me a joke")]])
```

## 사용자 정의 콜백 핸들러
##### 사용자 정의 콜백 핸들러를 생성하려면 콜백 핸들러가 처리할 이벤트와 이벤트가 트리거될 때 콜백 핸들러가 수행할 작업을 결정해야 한다.
##### 그 다음 콜백 핸들러를 생성자 콜백이나 요청 콜백으로 객체에 연결하면 된다.

```python
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"My custom handler, token: {token}")


prompt = ChatPromptTemplate.from_messages(["Tell me a joke about {animal}"])

model = ChatOpenAI(streaming=True, callbacks=[MyCustomHandler()])

chain = prompt | model

response = chain.invoke({"animal": "bears"})
```

## 파일 로깅
##### Langchain은 파일에 로그를 쓸 수 있는 FileCallbackHandler 기능을 제공한다.
##### StdOutCallbackHandler와 유사하지만 로그를 표준 출력으로 출력하는 대신 로그를 파일에 쓴다.

```python
from langchain_core.callbacks import FileCallbackHandler, StdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from loguru import logger

logfile = "output.log"

# 핸들러의 다른 출력을 기록한다.
logger.add(logfile, colorize=True, enqueue=True)
handler_1 = FileCallbackHandler(logfile)
handler_2 = StdOutCallbackHandler()

prompt = PromptTemplate.from_template("1 + {number} = ")
model = OpenAI()

# verbose flag에 따라, 표준 출력과 파일에 저장할 수도 있고, 파일에만 저장할 수 있다.
chain = prompt | model

response = chain.invoke({"number": 2}, {"callbacks": [handler_1, handler_2]})
logger.info(response)
```

##### 내용 확인 가능
```python
%pip install --upgrade --quiet  ansi2html > /dev/null

from ansi2html import Ansi2HTMLConverter
from IPython.display import HTML, display

with open("output.log", "r") as f:
    content = f.read()

conv = Ansi2HTMLConverter()
html = conv.convert(content, full=True)

display(HTML(html))
```

## 다중 콜백 핸들러
##### 이전까지는 "callbacks="를 사용하여 객체를 생성할 때 콜백 핸들러를 전달했는데, 해당 경우 콜백의 범위는 해당 특정 객체로 지정된다.
##### 실행할 때 arg 키워드를 CallbackHandlers 사용하여 전달하면 callbacks 실행과 관련된 모든 중첩 개체에서 해당 콜백이 실행된다.
##### 예로, 핸들러가 Agent를 통해 전달되면 에이전트와 관련된 모든 콜백 및 에이전트 실행과 관련된 모든 개체(Tools, LLMChain, LLM)에 사용된다.
##### 각각 개별 중첩 개체에 핸들러를 수동으로 연결할 필요가 없다!

```python
from typing import Any, Dict, List, Union

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.agents import AgentAction
from langchain_openai import OpenAI


# 첫번째로 사용자 정의 콜백 핸들러를 구현한다.
class MyCustomHandlerOne(BaseCallbackHandler):
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        print(f"on_llm_start {serialized['name']}")

    def on_llm_new_token(self, token: str, **kwargs: Any) -> Any:
        print(f"on_new_token {token}")

    def on_llm_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """Run when LLM errors."""

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> Any:
        print(f"on_chain_start {serialized['name']}")

    def on_tool_start(
        self, serialized: Dict[str, Any], input_str: str, **kwargs: Any
    ) -> Any:
        print(f"on_tool_start {serialized['name']}")

    def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
        print(f"on_agent_action {action}")


class MyCustomHandlerTwo(BaseCallbackHandler):
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        print(f"on_llm_start (I'm the second handler!!) {serialized['name']}")


# 핸들러들을 초기화한다.
handler1 = MyCustomHandlerOne()
handler2 = MyCustomHandlerTwo()

# agent를 정의한다.
llm = OpenAI(temperature=0, streaming=True, callbacks=[handler2])
tools = load_tools(["llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

# agent 실행한다.
agent.run("What is 2 raised to the 0.235 power?", callbacks=[handler1])
```

## 태그
##### call() / run() / apply() 메소드 tags에 인수를 전달하여 콜백에 태그를 추가할 수 있다.
##### 예로, 특정 LLMChain에 대한 모든 요청을 기록하려는 경우 태그를 추가한 다음 해당 태그를 기준으로 로그를 필터링할 수 있다. (생성자와 요청 콜백 모두 태그 전달 가능)
##### 이런 태그는 "start" 콜백 메소드의 인수로 전달된다.

## 토큰 계산
##### Langchain은 토큰을 계산할 수 있는 컨텍스트 관리자를 제공한다.

```python
import asyncio

from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
with get_openai_callback() as cb:
    llm.invoke("What is the square root of 4?")

total_tokens = cb.total_tokens
assert total_tokens > 0

with get_openai_callback() as cb:
    llm.invoke("What is the square root of 4?")
    llm.invoke("What is the square root of 4?")

assert cb.total_tokens == total_tokens * 2

# 컨텍스트 관리자 내에서 동시 실행을 할 수 있다.
with get_openai_callback() as cb:
    await asyncio.gather(
        *[llm.agenerate(["What is the square root of 4?"]) for _ in range(3)]
    )

assert cb.total_tokens == total_tokens * 3

# 컨텍스트 관리자의 동시성 안정성 가짐
task = asyncio.create_task(llm.agenerate(["What is the square root of 4?"]))
with get_openai_callback() as cb:
    await llm.agenerate(["What is the square root of 4?"])

await task
assert cb.total_tokens == total_tokens
```